<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Одномерное стационарное уравнение теплопроводности

### Examples - Thermodynamics
<section class="post-meta">
By Andreas S. Krogen, Jonas Tjemsland and Jon Andreas Støvneng.
</section>
Last edited: March 16th 2018 

This notebook is based on an assignment given in the course TMA4320 Introduction to Scientific Computation at NTNU, spring 2017.
___

## Вступление

Уравнение теплопроводности - это уравнение в частных производных, которое описывает распределение тепла в системе.
Оно может быть решено для того, чтобы найти колебания температуры в физическом теле.
В этом блокноте мы сначала выведем уравнение теплопроводности, описывающее систему, содержащую источник тепла.
Мы сосредоточимся на стационарном решении, когда система стабилизируется и нет зависимости от времени.
В качестве конкретного примера мы рассмотрим тонкий стержень с фиксированной температурой на концах. Стержень достаточно тонкий, чтобы его можно было считать одномерным, и нагревается гауссовым источником тепла.
Стационарное уравнение теплопроводности будет решаться с использованием метода коллокации, аппроксимирующего решение в виде суммы полиномов Лагранжа.

## Нестрогий вывод уравнения теплопроводности

Рассмотрим однородную систему с фиксированным объемом $\mathcal{V}$ и отсутствием обмена частицами с окружающей средой.

Пусть $u(\mathbf{r}, t)$ обозначает плотность энергии системы, $\mathbf{j}(\mathbf{r}, t)$ тепловой поток, выходящий из системы в окружающую среду, и пусть $\dot{q}(\mathbf{r}, t)$ обозначает тепло, поступающее в систему из источника на единицу объема в единицу времени (скорость нагрева на единицу объема). Изменение общей энергии системы должно быть равно энергии, поступающей в систему от источника тепла, за вычетом энергии, вытекающей из системы, следовательно

\begin{align*}
\frac{\mathrm{d}}{\mathrm{d}t} \int_\mathcal{V} u(\mathbf{r}, t) \,\mathrm{d}V &= \int_\mathcal{V} \dot{q}(\mathbf{r}, t) \,\mathrm{d}V - \oint_{\partial\mathcal{V}} \mathbf{j}(\mathbf{r}, t) \cdot \mathrm{d}\mathbf{S} \\
\int_\mathcal{V} \frac{\mathrm{\partial}}{\mathrm{\partial}t} u(\mathbf{r}, t) \,\mathrm{d}V &= \int_\mathcal{V} \dot{q}(\mathbf{r}, t) \,\mathrm{d}V - \int_\mathcal{V} \nabla\cdot \mathbf{j}(\mathbf{r}, t) \,\mathrm{d}V \\
\int_\mathcal{V} \left[ \frac{\mathrm{\partial}}{\mathrm{\partial}t} u(\mathbf{r}, t) + \nabla\cdot \mathbf{j}(\mathbf{r}, t) - \dot{q}(\mathbf{r}, t) \right] \,\mathrm{d}V &= 0
\end{align*}

$$\implies \frac{\mathrm{\partial}}{\mathrm{\partial}t} u(\mathbf{r}, t) + \nabla\cdot \mathbf{j}(\mathbf{r}, t) - \dot{q}(\mathbf{r}, t) = 0 \quad .$$

Мы предположили, что объем системы постоянен, что означает, что не требуется никакой объемной работы под давлением.
Тогда первый закон термодинамики гласит

\begin{align*}
\mathrm{d}U &= \mathrm{d}Q \\
            &= C_V \mathrm{d}T \quad ,
\end{align*}

где $U$ - внутренняя энергия, $T$ - температура системы и $C_V$ - теплоемкость системы при постоянном объеме.
Таким образом, частная производная по времени от внутренней плотности энергии $u$ может быть выражена как

$$\frac{\mathrm{\partial}u}{\mathrm{\partial}t} = \rho c_V \frac{\mathrm{\partial}T}{\mathrm{\partial}t} \quad .$$

Здесь $c_V$ - это теплоемкость на единицу массы, а $\rho$ - массовая плотность системы.

Закон Фурье - это эмпирический закон, который гласит, что тепловой поток пропорционален отрицательному градиенту температуры (тепловые потоки из областей с более высокой температурой в области с более низкой температурой),
$$ \mathbf{j} = - \kappa \, \nabla T \quad .$$

Постоянная $\kappa$ называется *теплопроводностью*.

Подстановка этих двух выражений в уравнение сохранения энергии дает уравнение теплоты


$$\frac{\mathrm{\partial}}{\mathrm{\partial}t} T(\mathbf{r}, t) = \frac{\kappa}{\rho c_V} \nabla^2 T(\mathbf{r}, t) + \frac{1}{\rho c_V} \, \dot{q}(\mathbf{r}, t) \quad .$$

Нас интересует стационарное решение, и никакая явная зависимость от времени не подразумевает, что

\begin{align*}
0                        &= \frac{\kappa}{\rho c_V} \nabla^2 T(\mathbf{r}) + \frac{1}{\rho c_V} \, \dot{q}(\mathbf{r}) \\
-\nabla^2 T(\mathbf{r}) &= \frac{1}{\kappa} \, \dot{q}(\mathbf{r}) \quad .
\end{align*}

В одномерном случае выходит

$$-\frac{\mathrm{d}^2}{\mathrm{d}x^2} T(x) = \frac{1}{\kappa} \, \dot{q}(x) \quad .$$

Единицы СИ соответствующих физических величин могут быть найдены с помощью размерного анализа:

Мы знаем, что $[T] = \mathrm{K}$ и $[\dot{q}] = \mathrm{Jm^{-3}s^{-1}}$, и хотим найти единицу $\kappa$.
Из одномерного уравнения теплопроводности мы имеем, что

\begin{align*}
[\kappa]      &= [\dot{q}(x)]\, \cdot \left[\frac{\mathrm{d}^2}{\mathrm{d}x^2} T(x)\right]^{-1} \\
              &= \mathrm{Jm^{-3}s^{-1}} \cdot (\mathrm{K m^{-2}})^{-1} \\
              &= \mathrm{JK^{-1}m^{-1}s^{-1}} \quad .
\end{align*}

## Пример: Моделирование тонкого металлического стержня, нагреваемого свечой

Рассмотрим тонкий металлический стержень длиной $L = b - a\,$ и площадью поперечного сечения $A$, которые считаются постоянными.
Стержень нагревается горящей свечой, которая моделируется гауссовым источником тепла

$$
\dot{q}(x) = \dot{q}_0 \exp\left[-\frac{1}{2}\left(\frac{x-x_0}{\sigma}\right)^2\right] \quad ,
$$

где $\dot{q}_0$ связано с мощностью источника тепла $P_\mathrm{src}$, $A$ и $\sigma$, что будет объяснено позже.
Пара тепловых резервуаров с постоянными температурами $T(a)$ и $T(b)$ поддерживают температуру стержня фиксированной на концах. См. Рис. 1.
<img src="images/heat_equation_sketch.png" width="500">

*Рисунок 1: Эскиз, изображающий физическую ситуацию. Концы металлического стержня прикреплены к тепловым резервуарам с температурами $T(a)$ и $T(b)$.*


Мы начнем с переписывания уравнения теплопроводности в более поддающуюся численному анализу форму

\begin{align*}
-\frac{\mathrm{d}^2}{\mathrm{d}x^2} T(x) &= \frac{\dot{q}_0}{\kappa} \exp\left[-\frac{1}{2}\left(\frac{x-x_0}{\sigma}\right)^2\right] \\
-\frac{\mathrm{d}^2}{\mathrm{d}x^2} \left[\frac{\kappa \, T(x)}{\dot{q}_0}\right] &= \exp\left[-\frac{1}{2}\left(\frac{x-x_0}{\sigma}\right)^2\right] \quad .
\end{align*}

Мы также вводим безразмерные переменные $\tilde{x} = (x-a)~/~L$ и $\tilde{\sigma} = \sigma~/~L$
сопоставление $a \mapsto 0$ и $b \mapsto 1$.
Это дает следующее безразмерное уравнение

\begin{align*}
-\frac{\mathrm{d}^2}{\mathrm{d}\tilde{x}^2} \left[\frac{\kappa \, T(\tilde{x})}{L^2 \, \dot{q_0}}\right] &= \exp\left[-\frac{1}{2}\left(\frac{\tilde{x}-\tilde{x}_0}{\tilde{\sigma}}\right)^2\right] \\
-\frac{\mathrm{d}^2}{\mathrm{d}\tilde{x}^2} \mathcal{T}(\tilde{x}) &= f(\tilde{x} ; \tilde{x}_0, \tilde{\sigma}) \quad .
\end{align*}


### Представление решения в базисе полиномов Лагранжа

Безразмерное уравнение теплопроводности может быть решено путем дискретизации интервала $[0,1]$ на оси $\tilde{x}$ и аппроксимации решения во всей области в виде суммы полиномов Лагранжа, требуя, чтобы аппроксимация удовлетворяла уравнению теплопроводности в каждой дискретной точке. Точки в области, в которых приближенное решение удовлетворяет уравнению теплопроводности, называются *точками коллокации*. Для краткого введения в интерполяцию Лагранжа, пожалуйста, взгляните на соответствующий раздел в нашего блокнота по [полиномиальной интерполяции](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/polynomial_interpolation.ipynb).
Интервал $[0,1]$ дискретизируется на $N$ точек, $0 = \tilde{x}_0 \lt \tilde{x}_1 \lt \dots \lt \tilde{x}_{N-2} \lt \tilde{x}_{N-1} = 1$.
Решение аппроксимируется

$$
\mathcal{T}(\tilde{x}) = \sum_{i = 0}^{N-1} c_i \mathrm{L}_i(\tilde{x}) \quad ,
$$

где 
$$
\mathrm{L}_i(\tilde{x}) \equiv \prod_{k = 0}^{N-1} \frac{\tilde{x} - \tilde{x}_k}{\tilde{x}_i - \tilde{x}_k}
$$

является полиномом Лагранжа, соответствующим точке $\tilde{x}_i$.
Многочлены Лагранжа обладают важным свойством $\mathrm{L}_i(\tilde{x}_j) = \delta_{ij}$.
Вставка полиномиального разложения в уравнение теплопроводности и применение этого свойства дает 

\begin{cases}
c_0  &= \mathcal{T}_a \\
-\sum_{j = 0}^{N-1} c_j \mathrm{L}_j''(\tilde{x}_i) &= f(\tilde{x}_i), & i = 1, \cdots, N-2 \\
c_{N-1} &= \mathcal{T}_b
\end{cases}

Эта система линейных уравнений может быть записана в матричном виде $A\vec{c} = \vec{f}$ :

$$
\begin{pmatrix}
1 & 0 & \cdots & 0 \\
-\mathrm{L}_0''(\tilde{x}_1) & -\mathrm{L}_1''(\tilde{x}_1) & \cdots & -\mathrm{L}_{N-1}''(\tilde{x}_1) \\
-\mathrm{L}_0''(\tilde{x}_2) & -\mathrm{L}_1''(\tilde{x}_2) & \cdots & -\mathrm{L}_{N-1}''(\tilde{x}_2) \\
\vdots & \vdots & \ddots & \vdots \\
-\mathrm{L}_0''(\tilde{x}_{N-2}) & -\mathrm{L}_1''(\tilde{x}_{N-2}) & \cdots & -\mathrm{L}_{N-1}''(\tilde{x}_{N-2}) \\
0 & 0 & \cdots & 1
\end{pmatrix}
\begin{pmatrix}
c_0 \\
c_1 \\
c_2 \\
\vdots \\
c_{N-2} \\
c_{N-1}
\end{pmatrix} =
\begin{pmatrix}
\mathcal{T}_a \\
f(\tilde{x}_1) \\
f(\tilde{x}_2) \\
\vdots \\
f(\tilde{x}_{N-2}) \\
\mathcal{T}_b
\end{pmatrix} \quad .
$$

Пусть $X = \{\tilde{x}_0, \tilde{x}_1, \cdots, \tilde{x}_{N-1}\}$. Вторая производная полиномов Лагранжа может быть выражена через

$$
\mathrm{L}_i''(\tilde{x}) = \sum_{\substack{x' \in X \setminus \{\tilde{x}_i\} \\ x'' \in X \setminus \{\tilde{x}_i, x'\}}} \frac{\mathrm{L}_i(\tilde{x})}{(\tilde{x} - x')(\tilde{x} - x'')} \quad .
$$

Это выражение становится легче вычислить численно, если рассматривать только $\tilde{x} \in X$. В этом случае выражение можно переписать, чтобы избежать деления на ноль.

Когда $i = j$,

$$
\mathrm{L}_i''(\tilde{x}_i) = \sum_{\substack{x' \in X \setminus \{\tilde{x}_i\} \\ x'' \in X \setminus \{\tilde{x}_i, x'\}}} \frac{2}{(\tilde{x}_i - x')(\tilde{x}_i - x'')} \quad .
$$

Когда $i \neq j$,

$$
\mathrm{L}_i''(\tilde{x}_j) = \frac{2}{\tilde{x}_i - \tilde{x}_j} \prod_{\, x' \in X \setminus \{ \tilde{x}_i, \tilde{x}_j \}} \left(\frac{\tilde{x}_j - x'}{\tilde{x}_i - x'}\right) \sum_{x'' \in X \setminus \{ \tilde{x}_i, \tilde{x}_j \}} \frac{1}{\tilde{x}_j - x''} \quad .
$$

Вывод выражения для второй производной полиномов Лагранжа оставлен в качестве упражнения для читателя (подсказка: логарифмическое дифференцирование).

Уравнение теплопроводности решается путем нахождения коэффициентов полиномов Лагранжа, что делается путем решения системы линейных уравнений, описанной выше.

### Реализация и проверка решателя

Теперь пришло время написать какой-то код. Мы начнем с реализации функций, которые оценивают полиномы Лагранжа и их вторые производные, а также функции, которые решают систему линейных уравнений и уравнение теплопроводности.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import itertools
from scipy.integrate import simps
import scipy.linalg.lapack as lapack

# Set some figure parameters
newparams = {
    'figure.figsize': (16, 6), 'axes.grid': True,
    'lines.linewidth': 1.5, 'font.size': 19, 'lines.markersize' : 10,
    'mathtext.fontset': 'stix', 'font.family': 'STIXGeneral'}
plt.rcParams.update(newparams)

def lagrangepoly(xarr, i, x):
    """ Evaluates the Lagrange polynomial L_i(x).
    
        Parameters:
            xarr: float array. Contains the points that define the particular set of Lagrange polynomials.
               i: int (positive). Selects the Lagrange polynomial corresponding to the point xarr[i].
               x: float. The point at which to evaluate the polynomial.
               
        Returns: The value of the Lagrange polynomial evaluated at x.
    """
    xs = np.delete(xarr, i)
    return np.prod(x - xs) / np.prod(xarr[i] - xs)

def lagrangepoly_2nd_deriv(xarr, i, j):
    """ Computes the second derivative of the Lagrange polynomial, L''_i(x_j).
        This function is limited to evaluating the second derivative at points contained in xarr only.
    
        Parameters:
            xarr: float array. Contains the points that define the particular set of Lagrange polynomials.
               i: int (positive). Selects the Lagrange polynomial corresponding to the point xarr[i].
               j: int (positive). Selects the point xarr[j] at which to evaluate the second derivative.
               
        Returns: The value of the second derivative of the Lagrange polynomial evaluated at xarr[j].
    """
    res = 0.
    if i == j:
        # Exclude element x[i] to avoid zero division
        xs = np.delete(xarr, [i])
        # Sum over all combinations of two elements in xs
        for p in itertools.combinations(xs, 2):
             res += 1. / ((xarr[j] - p[0])*(xarr[j] - p[1]))
    else:
        xs = np.delete(xarr, [i, j])
        res = 1. / (xarr[i] - xarr[j]) * np.prod(xarr[j] - xs) / np.prod(xarr[i] - xs) * np.sum(1. / (xarr[j] - xs))
    
    res *= 2
    return res

def compute_coefficients(xarr, farr):
    """ Computes the coefficients of the Lagrange polynomials defined by the points in xarr,
        essentially solving the heat equation.
        
        Parameters:
            xarr: float array. Contains the points that define the particular set of Lagrange polynomials.
            farr: float array. The values of the heat source term evaluated at the points in xarr.
           
        Returns: A float array containing the coefficients.
    """
    N = len(xarr)
    # Initialise matrix A (LHS)
    A = np.zeros((N, N), dtype=np.float)
    for i in range(1, N-1):
        for j in range(0, N):
            A[i, j] = -lagrangepoly_2nd_deriv(xarr, j, i)    
    A[0, 0] = 1.
    A[N-1, N-1] = 1.

    # Solve the system of linear equations
    carr = np.linalg.solve(A, farr)
    return carr

def solve_heat_equation(xs, N=40, T0=0.0, T1=0.0, x0=0.5, sigma=0.1, chebyshev=True):
    """ Solves the heat equation by expansion in a basis of Lagrange polynomials.
        
        Parameters:
            xs: float array. An array containing the points at which the solution should be evaluated.
             N: int (positive). The number of collocation points.
            T0: float. Temperature at x=0.
            T1: float. Temperature at x=1.
            x0: float. Mean of the Gaussian heat source.
         sigma: float. Standard deviation of the Gaussian heat source.
     chebyshev: boolean. Use chebyshev nodes as collocation points if true. Use linearly spaced points otherwise.
     
         Returns: A float array of the same dimension as xs containing the temperatures.
    """
    # Define collocation points.
    if chebyshev:
        xarr = 0.5 - 0.5*np.cos(np.arange(N)*np.pi/(N-1)) # Chebyshev nodes
    else:        
        xarr = np.linspace(0, 1, N) # Equally spaced points
    
    # Initialise farr containing the values of the heat source term (RHS)
    farr = np.exp(-0.5 * (xarr - x0)**2 / sigma**2)
    farr[0] = T0   # boundary conditions
    farr[N-1] = T1 #
    
    carr = compute_coefficients(xarr, farr)
    
    ntemps = len(xs)
    temps = np.zeros(ntemps)
    
    for i in range(0, N):
        for k in range(0, M):
            temps[k] += carr[i]*lagrangepoly(xarr, i, xs[k])
    return temps

Давайте проведем быстрый тест, в котором мы решим уравнение теплоты, когда температура на границе равна нулю, а источник тепла сосредоточен на $\tilde{x}=0.5$ .

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

M = 100
xs = np.linspace(0, 1, M)
ys = solve_heat_equation(xs) # Default parameters: N=40, T0=0.0, T1=0.0, x0=0.5, sigma=0.1
ax.plot(xs, ys)
ax.set_xlabel(r'$\tilde{x}$')
ax.set_ylabel(r'$\mathcal{T} \, (\tilde{x})$')
plt.show()

Температурная кривая гладкая, нулевая на границе и симметричная относительно $\tilde{x}=0,5$ . Таким образом, решение кажется правильным, по крайней мере, качественно, но нам нужно провести некоторые количественные тесты, чтобы убедиться, что результаты надежны. Общий способ сделать это - сравнить численное решение с известным решением в частном случае, который аналитически разрешим. В качестве примера мы выбираем аналитическое решение $\mathcal{T}(\tilde{x}) = \exp(\tilde{x})\cos(8\pi\tilde{x})$. Вставка этого решения в уравнение теплопроводности показывает,что соответствующий член источника равен$\, f(\tilde{x}) = \exp(\tilde{x})\left[(64\pi^2-1)\cos(8\pi\tilde{x}) + 16\pi\sin(8\pi\tilde{x})\right]$. Давайте решим уравнение теплопроводности с этим конкретным источником и построим результаты вместе с аналитическим решением для сравнения. Уравнение решается численно дважды: в первый раз с использованием точек с равным расстоянием в качестве точек коллокации, а затем с использованием узлов Чебышева, чтобы увидеть, есть ли какие-либо различия в ошибке.

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

# Number of collocation points
N = 20 

### Equally spaced collocation points
xarr_uniform = np.linspace(0, 1, N)

# Source term and boundary conditions
farr_uniform = np.exp(xarr_uniform)*((64*np.pi**2-1)*np.cos(8*np.pi*xarr_uniform)+16*np.pi*np.sin(8*np.pi*xarr_uniform))
farr_uniform[0] = 1
farr_uniform[N-1] = np.exp(1)

carr_uniform = compute_coefficients(xarr_uniform, farr_uniform)

ax1.plot(xarr_uniform, carr_uniform, 'rs')
ax1.plot(xarr_uniform, np.exp(xarr_uniform)*np.cos(8*np.pi*xarr_uniform), 'bx')
ax1.set_xlabel(r'$\tilde{x}$')
ax1.set_ylabel(r'$\mathcal{T} \, (\tilde{x})$')
ax1.set_xlim([-0.05, 1.2])
ax1.set_ylim([-3, 8])
ax1.legend(["Numerical", "Exact"], loc="right")
ax1.set_title("Equally spaced collocation points")

### Chebyshev nodes as collocation points
xarr_chebyshev = 0.5 - 0.5*np.cos(np.arange(N)*np.pi/(N-1))

# Source term and boundary conditions
farr_chebyshev = np.exp(xarr_chebyshev)*((64*np.pi**2-1)*np.cos(8*np.pi*xarr_chebyshev)+16*np.pi*np.sin(8*np.pi*xarr_chebyshev))
farr_chebyshev[0] = 1
farr_chebyshev[N-1] = np.exp(1)

carr_chebyshev = compute_coefficients(xarr_chebyshev, farr_chebyshev)

ax2.plot(xarr_chebyshev, carr_chebyshev, 'ro')
ax2.plot(xarr_chebyshev, np.exp(xarr_chebyshev)*np.cos(8*np.pi*xarr_chebyshev), 'bx')
ax2.set_xlabel(r'$\tilde{x}$')
ax2.set_ylabel(r'$\mathcal{T} \, (\tilde{x})$')
ax2.set_xlim([-0.05, 1.2])
ax2.set_ylim([-3, 8])
ax2.legend(["Numerical", "Exact"], loc="right")
ax2.set_title("Chebyshev nodes as collocation points")

fig.suptitle("Deviations from exact solution when N = {0}".format((N)), y=1.05)
plt.tight_layout()

plt.show()

Мы также строим ошибку в зависимости от количества точек коллокации $N$.
Ошибка определяется как наибольшее отклонение от точного решения в одной из точек коллокации,

$$
e_N = \max_{0 \leq i \leq N-1} \left|\mathcal{T}_{\mathrm{num}}(\tilde{x}_i) - \mathcal{T}_{\mathrm{exact}}(\tilde{x}_i)\right| \quad .
$$


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

Nstart = 5
Nstop = 60

Narr = np.arange(Nstart, Nstop+1, 5)
error_uniform = np.zeros(len(Narr))
error_chebyshev = np.zeros(len(Narr))

for i in range(0, len(Narr)):
    N = Narr[i]
    xarr_uniform = np.linspace(0, 1, N)
    farr_uniform = np.exp(xarr_uniform)*((64*np.pi**2-1)*np.cos(8*np.pi*xarr_uniform)+16*np.pi*np.sin(8*np.pi*xarr_uniform))
    farr_uniform[0] = 1
    farr_uniform[N-1] = np.exp(1)
    carr_uniform = compute_coefficients(xarr_uniform, farr_uniform)
    error_uniform[i] = np.amax(np.abs(carr_uniform - np.exp(xarr_uniform)*np.cos(8*np.pi*xarr_uniform)))
    
    xarr_chebyshev = 0.5 - 0.5*np.cos(np.arange(N)*np.pi/(N-1))
    farr_chebyshev = np.exp(xarr_chebyshev)*((64*np.pi**2-1)*np.cos(8*np.pi*xarr_chebyshev)+16*np.pi*np.sin(8*np.pi*xarr_chebyshev))
    farr_chebyshev[0] = 1
    farr_chebyshev[N-1] = np.exp(1)
    carr_chebyshev = compute_coefficients(xarr_chebyshev, farr_chebyshev)
    error_chebyshev[i] = np.amax(np.abs(carr_chebyshev - np.exp(xarr_chebyshev)*np.cos(8*np.pi*xarr_chebyshev)))

ax.set_title(r"Error as a function of the number of collocation points $N$")
ax.semilogy(Narr, error_uniform, 'rs')
ax.semilogy(Narr, error_chebyshev, 'bo')
ax.legend(["Uniform", "Chebyshev"])
ax.set_xlabel(r"$N$")
ax.set_ylabel(r"$e_N$")

plt.show()

Обратите внимание, что ошибка, как правило, больше для одинаково расположенных точек.
Это связано с так называемым феноменом Рунге и является причиной, по которой мы предпочитаем использовать узлы Чебышева, которые минимизируют ошибку. Для получения более подробной информации мы еще раз обратимся к нашему блокноту по [полиномиальной интерполяции](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/polynomial_interpolation.ipynb). График ошибок предполагает, что количество точек $N = 40$ должно быть достаточным, чтобы сохранить ошибку на низком уровне. Также обратите внимание на странное поведение, когда ошибка при использовании узлов Чебышева внезапно выравнивается, а ошибка при использовании равнорасположенных точек коллокации начинает расти. В случае равнорасположенных точек можно было бы ожидать, что ошибка будет расти с увеличением $N$ из-за явления Рунге, но ошибка при использовании узлов Чебышева теоретически должна продолжать уменьшаться. Причина, по которой ошибка Чебышева больше не уменьшается, заключается в ограниченной точности с плавающей запятой, доступной в подпрограмме, которая решает систему линейных уравнений. Функция `numpy.linalg.solve` вызывает базовую процедуру `LAPACK` `dgesv`, которая использует плавающие точки с двойной точностью, и это является ограничивающим фактором в данном случае.

### Нахождение температурной кривой металлического стержня

Теперь, когда мы более уверены в надежности решателя, мы можем попытаться решить реальную физическую задачу.
Предположим, что металлический стержень сделан из стали:

- Теплопроводность стального стержня предполагается постоянной $\kappa = 43~\mathrm{W\,K^{-1}m^{-1}}$. [2]
- Длина стержня составляет $L = 50~\mathrm{cm}$.
- Площадь поперечного сечения стержня равна $A = 1,0~\mathrm{cm}^2$.
- Приблизительная оценка мощности, которую типичная свеча будет генерировать при сгорании, составляет $P_\mathbf{src} \approx 85~\mathrm{W}$. См.Приложение.
- Стандартное отклонение распределения гауссова источника тепла устанавливается равным $\sigma = 1~\mathrm{cm}$.
- Температура на концах установлена на $T(a) = T(b) = 20~^\circ\mathrm{C}$.

Соответствующие безразмерные величины вычисляются в приведенном ниже коде и затем подаются в решатель.
Необходимо найти выражение, связывающее амплитуду $\dot{q}_0$ Гаусса с известными физическими величинами.
Общая мощность, поглощаемая стержнем от источника тепла, считывается

\begin{align*}
P_\mathbf{rod} &= \int_{a}^b \dot{q}(x) \, A \, \mathrm{d}x \\
               &\approx \int_{-\infty}^\infty \dot{q}(x) \, A \, \mathrm{d}x \\
               &= \dot{q}_0 A \int_{-\infty}^\infty \exp\left[-\frac{1}{2}\left(\frac{x-x_0}{\sigma}\right)^2\right] \mathrm{d}x \\
               &= \dot{q}_0 A \cdot \sqrt{2\pi}\sigma \quad .
\end{align*}

Интегральная аппроксимация хороша при условии, что значения Гаусса очень малы вне интервала $[a, b]$. 

Если мы предположим, что $20\%$ тепла, излучаемого источником, поглощается стержнем, то

$$
\dot{q}_0 = \frac{0.2 \cdot P_\mathbf{src}}{A \sqrt{2\pi} \sigma} \quad .
$$

После решения безразмерного уравнения теплопроводности фактические физические величины извлекаются следующим образом

\begin{align*}
T(\tilde{x}) &= \frac{\dot{q}_0}{\kappa} L^2 \cdot \mathcal{T}(\tilde{x}) \\
x &= L \cdot \tilde{x} \quad .
\end{align*}

Также вычисляется тепловой поток, который входит и выходит из системы. В случае стационарного теплового потока тепло, поступающее в систему, должно быть равно теплу, выходящему из системы, и это может служить тестом, определяющим, являются ли результаты физически обоснованными. Если энергия не сохраняется, что-то определенно подозрительно!

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

# Physical quantities
L = 0.5
A = 1e-04
kappa = 43
P_src = 85
heating_efficiency = 0.2 # The ratio of the power absorbed by the rod to the total power of the heat source
sigma = 0.01
q0 = heating_efficiency * P_src / (A*sigma*np.sqrt(2*np.pi))
x0 = L / 2
Ta = 20 + 273.15
Tb = 20 + 273.15

# Corresponding dimensionless quantities
sigma_dimless = sigma/L
T0 = Ta * kappa / (L**2*q0)
T1 = Tb * kappa / (L**2*q0)
x0_dimless = x0/L

N = 50
M = 1000
xs_dimless = np.linspace(0, 1, M)
temps_dimless = solve_heat_equation(xs_dimless, N, T0, T1, x0_dimless, sigma_dimless)
xs_physical = xs_dimless*L
temps_physical = temps_dimless*L**2*q0/kappa - 273.15 # in degrees Celcius


ax.plot(xs_physical, temps_physical)
ax.set_xlabel(r'$x \quad [\mathrm{m}]$')
ax.set_ylabel(r'$T \quad [\mathrm{^\circ C}]$')
ax.set_title("The temperature profile of the steel rod")
plt.show()

# Compute average temperature (Simpson's method)
avg_temp_physical = simps(temps_physical, xs_physical) / L
print("Average temperature (celcius): {:.2f}".format(avg_temp_physical))

# Compute heat flux out of and into the system
xarr = np.linspace(0, L, M)
distr = np.exp(-0.5 * (xarr - x0)**2 / sigma**2)
heat_flux_from_candle = q0*simps(distr, xarr) * A
print("Heat flux from the candle (W): \t{:.2f}".format(heat_flux_from_candle))
delta_x = L / M
heat_flux_out = kappa*(temps_physical[1] - temps_physical[0] + temps_physical[-2] - temps_physical[-1]) / delta_x * A
print("Heat flux out of system (W): \t{:.2f}".format(heat_flux_out))

Тепловой поток из системы равен тепловому потоку в систему, следовательно, энергия сохраняется.
Из графика видно, что стальной стержень становится довольно горячим! Однако стоит отметить, что мы не включили в модель никаких слагаемых, описывающих потери тепла от стержня в окружающую среду. При этой конкретной обработке единственным способом, с помощью которого стержень может терять энергию, является тепло, поступающее в резервуары на концах. Поэтому можно было бы ожидать, что в действительности температура будет несколько ниже.

### Предлагаемые упражнения и улучшения

- Решите зависящее от времени уравнение теплопроводности с использованием тех же параметров и визуализируйте временную эволюцию температурного профиля.
- Найти способ моделирования потерь тепла от стержня в окружающую среду за пределами тепловых резервуаров.
- Изучите количество времени, необходимое для достижения стационарного решения в различных сценариях.

## Appendix

### Оценка тепла, выделяемого горящей свечой из парафина

Предположим, что свеча имеет цилиндрическую форму с радиусом $r = 1,0~\mathrm{cm}$ и высотой $h = 30~\mathrm{cm}$.
Массовая плотность парафина примерно равна
$\rho = 0,9~\mathrm{g\,cm^{-3}}$ и
теплота сгорания равна $q_\mathrm{cb} = 43,1~\mathrm{kJ\,g^{-1}}$. [3, 4]
Если мы предположим, что время горения $T = 12~\mathrm{hours}$, тепло, выделяемое свечой в единицу времени, равно

\begin{align*}
P &= \rho \cdot \pi r^2 h \cdot q_\mathrm{cb}~/~T \\
          &\approx 85~\mathrm{W} \quad .
\end{align*}

### References

[1] Description of project 1 (Norwegian), TMA4320 Introduction to Scientific Computation, NTNU, spring 2017, Available: https://wiki.math.ntnu.no/_media/tma4320/2017v/project_spectral_korrigert.pdf (accessed: 06.04.2017).

[2] The Engineering ToolBox. "Thermal Conductivity of common Materials and Gases". Available: http://www.engineeringtoolbox.com/thermal-conductivity-d_429.html (accessed 20.04.2017).

[3] Kaye, G. W. C., Laby, T. H. "Mechanical properties of materials". Kaye and Laby Tables of Physical and Chemical Constants. National Physical Laboratory. Available: http://www.kayelaby.npl.co.uk/general_physics/2_2/2_2_1.html (accessed: 20.04.2017).

[4] Dillon, S. E., Hamins, A. "Ignition propensity and heat flux profiles of candle flames for fire investigation". Fire and Materials 2003. 8th International Conference. Conference Papers (2003).
Available: http://www.fire.nist.gov/bfrlpubs/fire03/PDF/f03032.pdf (accessed: 20.04.2017).